Importing and Loading data

In [10]:
import pandas as pd
import numpy as np
import time
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress ConvergenceWarning from LinearSVC for cleaner output
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# --- 1. Load Data and Prepare X and y (Training) ---
df_train = pd.read_csv("train_processed_parv.csv")
bool_cols_train = df_train.select_dtypes(include='bool').columns
df_train[bool_cols_train] = df_train[bool_cols_train].astype(int)

X = df_train.drop(columns=['ProfileID', 'RiskFlag'])
y = df_train['RiskFlag']

# Split for full training (80%) and evaluation (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Create the 20% Training Sample (25% of X_train_full = 20% of original data)
X_train_20pct, X_temp, y_train_20pct, y_temp = train_test_split(
    X_train_full, y_train_full, test_size=0.75, random_state=42, stratify=y_train_full
)

input_dim = X_train_full.shape[1]

print(f"Total Features: {input_dim}")
print(f"20% Training Sample Size: {len(X_train_20pct)}")
print(f"Full Training Set Size: {len(X_train_full)}")
print(f"Evaluation Test Set Size: {len(X_test)}\n")

# --- 2. Load and Prepare X for Submission (Competition Test Data) ---
df_test = pd.read_csv("test_processed_parv.csv")
bool_cols_test = df_test.select_dtypes(include='bool').columns
df_test[bool_cols_test] = df_test[bool_cols_test].astype(int)

# X_submit is the actual competition test data to predict
X_submit = df_test.drop(columns=['ProfileID'])
profile_ids = df_test['ProfileID']
print(f"Submission Test Set Size: {len(X_submit)}\n")


# --- 3. Define Common Functions ---

def evaluate_model(model_name, model, X_train, y_train, X_eval, y_eval, is_ann=False):
    """Trains, predicts, and reports metrics for a given model."""
    print(f"--- Training {model_name} on {len(X_train)} samples... ---")
    start_time = time.time()

    if is_ann:
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)

    training_time = time.time() - start_time
    print(f"Training Time: {training_time:.2f} seconds")

    # Predict on the hold-out evaluation set
    if is_ann:
        y_pred_proba = model.predict(X_eval, verbose=0)
        y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    else:
        y_pred = model.predict(X_eval)

    accuracy = accuracy_score(y_eval, y_pred)
    auc = roc_auc_score(y_eval, y_pred)

    print(f"Evaluation Accuracy: {accuracy:.4f}")
    print(f"Evaluation ROC AUC: {auc:.4f}")
    print("\nClassification Report (on Evaluation Set):")
    print(classification_report(y_eval, y_pred, zero_division=0))
    print("-" * 50)
    return model # Return the trained model

def create_ann_model(input_dim):
    """Creates a basic ANN model for binary classification."""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

Total Features: 26
20% Training Sample Size: 40855
Full Training Set Size: 163421
Evaluation Test Set Size: 40856

Submission Test Set Size: 51070



Common Functions and Neural Networks

In [11]:
# --- 3. Define Common Functions ---

def evaluate_model(model_name, model, X_train, y_train, X_eval, y_eval, is_ann=False):
    """Trains, predicts, and reports metrics for a given model."""
    print(f"--- Training {model_name} on {len(X_train)} samples... ---")
    start_time = time.time()

    if is_ann:
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)

    training_time = time.time() - start_time
    print(f"Training Time: {training_time:.2f} seconds")

    # Predict on the hold-out evaluation set
    if is_ann:
        y_pred_proba = model.predict(X_eval, verbose=0)
        y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    else:
        y_pred = model.predict(X_eval)

    accuracy = accuracy_score(y_eval, y_pred)
    auc = roc_auc_score(y_eval, y_pred)

    print(f"Evaluation Accuracy: {accuracy:.4f}")
    print(f"Evaluation ROC AUC: {auc:.4f}")
    print("\nClassification Report (on Evaluation Set):")
    print(classification_report(y_eval, y_pred, zero_division=0))
    print("-" * 50)
    return model # Return the trained model

def create_ann_model(input_dim):
    """Creates a basic ANN model for binary classification."""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

Linear SVM

In [12]:
# --- A. Linear SVM (Training on 20% Sample) ---
linear_svm_20pct = LinearSVC(random_state=42, dual=True, max_iter=10000)
evaluate_model("Linear SVM (20%)", linear_svm_20pct,
               X_train_20pct, y_train_20pct, X_test, y_test)

# --- B. Linear SVM (Training on Full Dataset) ---
final_linear_svm = LinearSVC(random_state=42, dual=True, max_iter=10000)
final_linear_svm = evaluate_model("Linear SVM (Full)", final_linear_svm,
                                  X_train_full, y_train_full, X_test, y_test)


# --- C. Prediction and Submission ---
print("Predicting with Linear SVM (Full) on Test Data...")
y_pred_linear = final_linear_svm.predict(X_submit)

# Create and save the submission DataFrame
submission_df_linear = pd.DataFrame({
    'ProfileID': profile_ids,
    'RiskFlag': y_pred_linear
})
submission_filename_linear = 'submission_linear_svm.csv'
submission_df_linear.to_csv(submission_filename_linear, index=False)
print(f"Generated submission file: {submission_filename_linear}")

--- Training Linear SVM (20%) on 40855 samples... ---
Training Time: 7.26 seconds
Evaluation Accuracy: 0.8837
Evaluation ROC AUC: 0.5002

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     36105
           1       0.50      0.00      0.00      4751

    accuracy                           0.88     40856
   macro avg       0.69      0.50      0.47     40856
weighted avg       0.84      0.88      0.83     40856

--------------------------------------------------
--- Training Linear SVM (Full) on 163421 samples... ---
Training Time: 69.59 seconds
Evaluation Accuracy: 0.8837
Evaluation ROC AUC: 0.5002

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     36105
           1       0.67      0.00      0.00      4751

    accuracy                           0.88     40856
   macro avg       0.78      0.50      

Kernalized SVM

In [ ]:
# --- A. Kernelized SVM (RBF) (Training on 20% Sample) ---
# NOW TRAINING ON THE ENTIRE 20% SAMPLE (X_train_20pct)
rbf_svm_20pct = SVC(kernel='rbf', gamma='scale', random_state=42)
rbf_svm_20pct = evaluate_model("Kernelized SVM (RBF) (20% full sample)", rbf_svm_20pct,
               X_train_20pct, y_train_20pct, X_test, y_test)


# --- B. Kernelized SVM (RBF) (Training on Full Dataset) ---
# NOW TRAINING ON THE ENTIRE FULL TRAINING DATASET (X_train_full)
final_rbf_svm = SVC(kernel='rbf', gamma='scale', random_state=42)
final_rbf_svm = evaluate_model("Kernelized SVM (RBF) (Full Dataset)", final_rbf_svm,
                               X_train_full, y_train_full, X_test, y_test)


# --- C. Prediction and Submission ---
print("Predicting with Kernelized SVM (Full Dataset) on Test Data...")
y_pred_rbf = final_rbf_svm.predict(X_submit)

# Create and save the submission DataFrame
submission_df_rbf = pd.DataFrame({
    'ProfileID': profile_ids,
    'RiskFlag': y_pred_rbf
})
submission_filename_rbf = 'submission_kernelized_svm.csv'
submission_df_rbf.to_csv(submission_filename_rbf, index=False)
print(f"Generated submission file: {submission_filename_rbf}")

MLP (Neural Networks)

In [13]:
# --- A. ANN (Training on 20% Sample) ---
ann_model_20pct = create_ann_model(input_dim)
evaluate_model("ANN (20%)", ann_model_20pct,
               X_train_20pct, y_train_20pct, X_test, y_test, is_ann=True)


# --- B. ANN (Training on Full Dataset) ---
final_ann_model = create_ann_model(input_dim)
final_ann_model = evaluate_model("ANN (Full)", final_ann_model,
                                 X_train_full, y_train_full, X_test, y_test, is_ann=True)


# --- C. Prediction and Submission ---
print("Predicting with ANN (Full) on Test Data...")
# Predict probabilities
y_pred_proba_ann = final_ann_model.predict(X_submit, verbose=0)
# Convert probabilities to binary predictions (0 or 1)
y_pred_ann = (y_pred_proba_ann > 0.5).astype(int).flatten()

# Create and save the submission DataFrame
submission_df_ann = pd.DataFrame({
    'ProfileID': profile_ids,
    'RiskFlag': y_pred_ann
})
submission_filename_ann = 'submission_mlp.csv'
submission_df_ann.to_csv(submission_filename_ann, index=False)
print(f"Generated submission file: {submission_filename_ann}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Training ANN (20%) on 40855 samples... ---
Training Time: 26.20 seconds
Evaluation Accuracy: 0.8849
Evaluation ROC AUC: 0.5230

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     36105
           1       0.55      0.05      0.09      4751

    accuracy                           0.88     40856
   macro avg       0.72      0.52      0.52     40856
weighted avg       0.85      0.88      0.84     40856

--------------------------------------------------
--- Training ANN (Full) on 163421 samples... ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training Time: 104.19 seconds
Evaluation Accuracy: 0.8850
Evaluation ROC AUC: 0.5276

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     36105
           1       0.55      0.06      0.11      4751

    accuracy                           0.89     40856
   macro avg       0.72      0.53      0.52     40856
weighted avg       0.85      0.89      0.84     40856

--------------------------------------------------
Predicting with ANN (Full) on Test Data...
Generated submission file: submission_ann.csv


Logistic Regression

In [14]:


# --- A. Logistic Regression (Training on 20% Sample) ---
# Use max_iter=1000 for convergence on large datasets
log_reg_20pct = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)
evaluate_model("Logistic Regression (20%)", log_reg_20pct,
               X_train_20pct, y_train_20pct, X_test, y_test)

# --- B. Logistic Regression (Training on Full Dataset) ---
final_log_reg = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)
final_log_reg = evaluate_model("Logistic Regression (Full)", final_log_reg,
                               X_train_full, y_train_full, X_test, y_test)


# --- C. Prediction and Submission ---
print("Predicting with Logistic Regression (Full) on Test Data...")
y_pred_log_reg = final_log_reg.predict(X_submit)

# Create and save the submission DataFrame
submission_df_log_reg = pd.DataFrame({
    'ProfileID': profile_ids,
    'RiskFlag': y_pred_log_reg
})
submission_filename_log_reg = 'submission_logistic_regression.csv'
submission_df_log_reg.to_csv(submission_filename_log_reg, index=False)
print(f"Generated submission file: {submission_filename_log_reg}")

--- Training Logistic Regression (20%) on 40855 samples... ---
Training Time: 0.20 seconds
Evaluation Accuracy: 0.8842
Evaluation ROC AUC: 0.5209

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     36105
           1       0.52      0.05      0.09      4751

    accuracy                           0.88     40856
   macro avg       0.71      0.52      0.51     40856
weighted avg       0.85      0.88      0.84     40856

--------------------------------------------------
--- Training Logistic Regression (Full) on 163421 samples... ---
Training Time: 1.13 seconds
Evaluation Accuracy: 0.8843
Evaluation ROC AUC: 0.5204

Classification Report (on Evaluation Set):
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     36105
           1       0.53      0.05      0.09      4751

    accuracy                           0.88     40856
   macro avg       0.7